<a href="https://colab.research.google.com/github/Iremguel/Fallstudie_Elektrifizierung_der_Logistik/blob/main/Stufe_2_(ohne_Ladeinfrastruktur).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Elektrifizierung

In [291]:
! pip install -q pyscipopt

In [292]:
import pandas as pd
from pyscipopt import Model, quicksum

In [293]:
! git clone https://github.com/Iremguel/Fallstudie_Elektrifizierung_der_Logistik.git

fatal: destination path 'Fallstudie_Elektrifizierung_der_Logistik' already exists and is not an empty directory.


In [294]:
! ls

Fallstudie_Elektrifizierung_der_Logistik  sample_data


In [295]:
! ls Daten

ls: cannot access 'Daten': No such file or directory


In [296]:
folder = "Fallstudie_Elektrifizierung_der_Logistik/Daten/"

In [297]:
chargers = pd.read_csv(f"{folder}chargers.csv", sep=";")

In [298]:
diesel_trucks = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [299]:
electric_trucks = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [300]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

## Hilfunktionen

In [301]:
def time_to_slot(t):
    """Wandelt HH:MM in 15-Minuten-Zeitschritte um"""
    h, m = map(int, t.split(":"))
    return (60 * h + m) // 15

routes["start"] = routes["starttime"].apply(time_to_slot)
routes["end"]   = routes["endtime"].apply(time_to_slot)

## Indexmengen

In [302]:
R = list(routes["route_id"].unique()) # Routen

#LKWs
#K_D = list(diesel_trucks["truck_model"].unique()) # Diesel LKW-Typen
K_E = list(electric_trucks["truck_model"].unique()) # e-LKW Typen
#K = K_D + K_E # Gesamte LKW-Flotte

V = list(range(len(R)))      # physische LKWs (Ausprägung)

S = list(chargers["charger_model"].unique()) # Ladesäulentypen

T = list(range(96))  # 24h * 4

In [303]:
DAYS = list(range(260))

## Parameter

In [304]:
# Diesel-LKWs
#diesel_capex = diesel_trucks.set_index("truck_model")["capex_yearly"].to_dict()
#diesel_opex  = diesel_trucks.set_index("truck_model")["opex_yearly"].to_dict()
#diesel_kfz   = diesel_trucks.set_index("truck_model")["kfz_yearly"].to_dict()

#diesel_consumption = diesel_trucks.set_index("truck_model")["avg_diesel_per_100km"].to_dict()

In [305]:
# Diesel-LKW-Parameter (nur ein Typ)
d = "ActrosL"

diesel_capex = diesel_trucks.set_index("truck_model").loc[truck, "capex_yearly"]
diesel_opex  = diesel_trucks.set_index("truck_model").loc[truck, "opex_yearly"]
diesel_kfz   = diesel_trucks.set_index("truck_model").loc[truck, "kfz_yearly"]
diesel_cons  = diesel_trucks.set_index("truck_model").loc[truck, "avg_diesel_per_100km"]

# Konstanten
diesel_price = 1.60  # €/Liter
toll_price   = 0.34  # €/km
DAYS = 260           # Betriebstage pro Jahr

# Fixkosten pro LKW (jährlich)
diesel_fixed_cost = diesel_capex + diesel_opex + diesel_kfz

In [306]:
# Tourenparameter
route_start = routes.set_index("route_id")["start"].to_dict()
route_end   = routes.set_index("route_id")["end"].to_dict()
distance_total = routes.set_index("route_id")["distance_total"].to_dict()
distance_toll  = routes.set_index("route_id")["distance_toll"].to_dict()

In [307]:
# e-LKWs
# Kosten und Erlöse
electric_capex = electric_trucks.set_index("truck_model")["capex_yearly"].to_dict()
electric_opex  = electric_trucks.set_index("truck_model")["opex_yearly"].to_dict()
thg_revenue    = electric_trucks.set_index("truck_model")["thg_yearly"].to_dict()

# Energie und Technik
electric_consumption = electric_trucks.set_index("truck_model")["avg_energy_kWh_per_100km"].to_dict()
truck_max_power      = electric_trucks.set_index("truck_model")["max_power"].to_dict()
soc_max              = electric_trucks.set_index("truck_model")["soc_max_kWh"].to_dict()
electricity_price = 0.25  # €/kWh

In [308]:
# Routenkosten pro Jahr
diesel_route_cost = {}
for r in R:
    fuel = diesel_cons / 100 * distance_total[r] * diesel_price
    toll = distance_toll[r] * toll_price
    diesel_route_cost[r] = DAYS * (fuel + toll)

electric_route_cost = {}
for r in R:
    for k in K_E:
        energy_cost = (
            electric_consumption[k] / 100
            * distance_total[r]
            * electricity_price
        )
        electric_route_cost[r, k] = DAYS * energy_cost

In [309]:
#Ladesäulen
charger_capex = chargers.set_index("charger_model")["capex_yearly"].to_dict()
charger_opex  = chargers.set_index("charger_model")["opex_yearly"].to_dict()
charger_power = chargers.set_index("charger_model")["max_power"].to_dict()
charger_spots = chargers.set_index("charger_model")["charging_spots"].to_dict()

In [310]:
# Diesel-Kosten pro Route & LKW
#diesel_cost = {}
#for r in R:
#    for k in K_D:
#        fuel_cost = (
#            diesel_consumption[k] / 100
#            * distance_total[r]
#            * diesel_price
#        )
#        toll_cost = distance_toll[r] * toll_price
#        diesel_cost[r, k] = fuel_cost + toll_cost


In [311]:
# Energiebedarf E-LKW pro Route
energy_need = {}
for r in R:
    for k in K_E:
        energy_need[r, k] = (
            electric_consumption[k] / 100
            * distance_total[r]
        )


In [312]:
# Energiepreise & Netz
energy_price = 0.25      # €/kWh
grid_fee     = 1000      # €/year
power_price  = 150       # €/kW/year

grid_base_capacity = 500
grid_extension     = 500
grid_extension_cost = 10000


In [313]:
# Batteriespeicher-Parameter
batt_capex_power = 30    # €/kW/year
batt_capex_energy = 350  # €/kWh/year
batt_opex_rate = 0.02
battery_efficiency = 0.98
battery_dod = 0.975

## Modellinstanz ziehen

In [314]:
model = Model()

## Entscheidungsvariablen

In [315]:
# Tour–LKW-Zuordnung
# x[r,v] = 1 wenn Route r von LKW v gefahren wird
X_D = {}
for r in R:
    for v in V:
        X_D[r, v] = model.addVar(vtype="B", name=f"xD_{r}_{v}")

X_E = {}
for r in R:
    for v in V:
      for k in K_E:
        X_E[r, v, k] = model.addVar(vtype="B", name=f"xE_{r}_{v}_{k}")

In [316]:
# y[v] = 1 wenn LKW v genutzt wird
Y_D = {}
for v in V:
    Y_D[v] = model.addVar(vtype="B", name=f"yD_{v}")

Y_E = {}
for v in V:
  for k in K_E:
    Y_E[v, k] = model.addVar(vtype="B", name=f"yE_{v}_{k}")

In [317]:
# Installation von Ladesäulen
#Y = {}
#for s in S:
#    Y[s] = model.addVar(vtype="B", name=f"y_{s}")

In [318]:
# Erweiterung durch Transformator
#Z = model.addVar(vtype="B", name="grid_extension")

In [319]:
# Batteriespeicher
#P_batt = model.addVar(lb=0, name="battery_power")     # kW
#E_batt = model.addVar(lb=0, name="battery_capacity") # kWh

## Flussvariablen

In [320]:
# Ladeleistung von E-LKW k im Zeitslot t
#p_charge_truck = {}
#for k in K_E:
#    for t in T:
#        p_charge_truck[k, t] = model.addVar(
#            lb=0,
#            name=f"p_charge_{k}_{t}"
#        )

In [321]:
# Laden des Batteriespeichers
#p_batt_charge = {}
#for t in T:
#    p_batt_charge[t] = model.addVar(
#        lb=0,
#        name=f"p_batt_charge_{t}"
#    )

In [322]:
# Entladen des Batteriespeichers
#p_batt_discharge = {}
#for t in T:
#    p_batt_discharge[t] = model.addVar(
#        lb=0,
#        name=f"p_batt_discharge_{t}"
#    )

In [323]:
# Netzbezugsleistung im Slot t
#p_grid = {}
#for t in T:
#    p_grid[t] = model.addVar(lb=0, name=f"p_grid_{t}")

In [324]:
# Optional: Energieverbrauch von Route r durch LKW k
#energy_need[r,k]

## Zielfunktion

In [325]:
model.setObjective(
    # Diesel Fixkosten
    quicksum(diesel_fixed_cost * Y_D[v] for v in V)
    +
    # e-LKW Fixkosten minus THG
    quicksum(
        (electric_capex[k] + electric_opex[k] - thg_revenue[k]) * Y_E[v, k]
        for v in V for k in K_E
    )
    +
    # Variable Kosten
    quicksum(diesel_route_cost[r] * X_D[r, v] for r in R for v in V)
    +
    quicksum
        (electric_route_cost[r, k] * X_E[r, v, k]
        for r in R for v in V for k in K_E
    )
    ,
    sense="minimize"

)

## Nebenbedingungen

In [326]:
# Jede Route wird genau einmal gefahren (Diesel ODER e-LKW)
for r in R:
    model.addCons(
        quicksum(X_D[r, v] for v in V)
        + quicksum(X_E[r, v, k] for v in V for k in K_E)
        == 1,
        name=f"route_coverage_{r}"
    )


In [327]:
# Route nur wenn LKW existiert
for r in R:
    for v in V:
        model.addCons(X_D[r, v] <= Y_D[v])
        for k in K_E:
            model.addCons(X_E[r, v, k] <= Y_E[v, k])

In [328]:
# Ein physischer LKW ist ENTWEDER Diesel ODER ein e-LKW-Typ
for v in V:
    model.addCons(
        Y_D[v] + quicksum(Y_E[v, k] for k in K_E) <= 1,
        name=f"one_vehicle_type_{v}"
    )

In [329]:
# Keine zeitliche Überlappung auf demselben LKW
for v in V:
    for i, r1 in enumerate(R):
        for r2 in R[i+1:]:
            if not (
                route_end[r1] <= route_start[r2]
                or route_end[r2] <= route_start[r1]
            ):
                model.addCons(
                    X_D[r1, v] + X_D[r2, v]
                    + quicksum(X_E[r1, v, k] + X_E[r2, v, k] for k in K_E)
                    <= 1,
                    name=f"overlap_{r1}_{r2}_{v}"
                )

In [330]:
# Ladeverbot während der Route
#for r in R:
#    for k in K_E:
#        for t in route_slots[r]:
#            model.addCons(
#                p_charge_truck[k, t] <= (1 - X[r, k]) * truck_max_power[k],
#                name=f"no_charge_during_route_{r}_{k}_{t}"
#            )

In [331]:
# Energiebedarf der Route muss geladen werden
#for r in R:
#    for k in K_E:
#        model.addCons(
#            quicksum(p_charge_truck[k, t] * 0.25 for t in T)
#            >= energy_need[r, k] * X[r, k],
#            name=f"energy_balance_{r}_{k}"
#        )

In [332]:
# Maximale Ladeleistung je e-LKW
#for k in K_E:
#    for t in T:
#        model.addCons(
#            p_charge_truck[k, t] <= truck_max_power[k],
#            name=f"truck_power_limit_{k}_{t}"
#        )


In [333]:
# Ladesäulen-Leistung nicht überschreiten
#for t in T:
#    model.addCons(
#        quicksum(p_charge_truck[k, t] for k in K_E)
#        <= quicksum(Y[s] * charger_power[s] for s in S),
#        name=f"charger_power_limit_{t}"
#    )

In [334]:
# Maximal 3 Ladesäulen
#model.addCons(
#    quicksum(Y[s] for s in S) <= 3,
#    name="max_chargers"
#)

In [335]:
# Netzanschlussleistung mit Transformator
#for t in T:
#    model.addCons(
#        p_grid[t] <= grid_base_capacity + Z * grid_extension,
#        name=f"grid_capacity_{t}"
#    )

In [336]:
# Leistungsbilanz (Netz + Batterie + Laden)
#for t in T:
#    model.addCons(
#        p_grid[t] + p_batt_discharge[t]
#        >= quicksum(p_charge_truck[k, t] for k in K_E)
#        + p_batt_charge[t],
#        name=f"power_balance_{t}"
#    )

In [337]:
# Batteriegrenzen
#for t in T:
#    model.addCons(p_batt_charge[t] <= P_batt)
#    model.addCons(p_batt_discharge[t] <= P_batt)

## Lösung

In [338]:
#model.redirectOutput()

In [339]:
model.optimize()

In [340]:
print("\n==============================")
print("OPTIMALE LÖSUNG (OHNE LADEN)")
print("==============================")
print(f"Gesamtkosten pro Jahr: {model.getObjVal():,.2f} €")

print("\nEingesetzte Fahrzeuge:")
for v in V:
    if model.getVal(Y_D[v]) > 0.5:
        print(f"LKW {v}: Diesel")
    for k in K_E:
        if model.getVal(Y_E[v, k]) > 0.5:
            print(f"LKW {v}: e-LKW {k}")

print("\nRoutenplanung:")
for r in R:
    for v in V:
        if model.getVal(X_D[r, v]) > 0.5:
            print(f"Route {r} → Diesel LKW {v}")
        for k in K_E:
            if model.getVal(X_E[r, v, k]) > 0.5:
                print(f"Route {r} → e-LKW {k} (LKW {v})")


OPTIMALE LÖSUNG (OHNE LADEN)
Gesamtkosten pro Jahr: 851,418.35 €

Eingesetzte Fahrzeuge:
LKW 3: Diesel
LKW 5: Diesel
LKW 8: Diesel
LKW 9: e-LKW eActros400
LKW 10: e-LKW eActros400
LKW 11: Diesel
LKW 12: e-LKW eActros400
LKW 13: e-LKW eActros400
LKW 14: Diesel
LKW 15: e-LKW eActros400
LKW 16: Diesel
LKW 17: Diesel
LKW 18: e-LKW eActros400
LKW 19: Diesel

Routenplanung:
Route t-4 → e-LKW eActros400 (LKW 10)
Route t-5 → e-LKW eActros400 (LKW 15)
Route t-6 → e-LKW eActros400 (LKW 18)
Route s-1 → e-LKW eActros400 (LKW 9)
Route s-2 → Diesel LKW 19
Route s-3 → e-LKW eActros400 (LKW 13)
Route s-4 → e-LKW eActros400 (LKW 12)
Route w1 → Diesel LKW 11
Route w2 → Diesel LKW 14
Route w3 → Diesel LKW 8
Route w4 → Diesel LKW 5
Route w5 → Diesel LKW 16
Route w6 → Diesel LKW 3
Route w7 → Diesel LKW 17
Route r1 → e-LKW eActros400 (LKW 18)
Route r2 → e-LKW eActros400 (LKW 13)
Route r3 → e-LKW eActros400 (LKW 12)
Route h3 → e-LKW eActros400 (LKW 15)
Route h4 → e-LKW eActros400 (LKW 10)
Route k1 → e-LKW e